In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fastai.text import *

In [ ]:
EOS = '<eos>'
PATH=Path('../data/wikitext')

Small helper function to read the tokens.

In [ ]:
def read_file(filename):
    tokens = []
    with open(PATH/filename, encoding='utf8') as f:
        for line in f:
            tokens.append(line.split() + [EOS])
    return np.array(tokens)

In [ ]:
trn_tok = read_file('wiki.train.tokens')
val_tok = read_file('wiki.valid.tokens')
tst_tok = read_file('wiki.test.tokens')

In [ ]:
len(trn_tok), len(val_tok), len(tst_tok)

In [ ]:
' '.join(trn_tok[4][:20])

In [ ]:
cnt = Counter(word for sent in trn_tok for word in sent)
cnt.most_common(10)

Give an id to each token and add the pad token (just in case we need it).

In [ ]:
itos = [o for o,c in cnt.most_common()]
itos.insert(0,'<pad>')

In [ ]:
vocab_size = len(itos); vocab_size

Creates the mapping from token to id then numericalizing our datasets.

In [ ]:
stoi = collections.defaultdict(lambda : 5, {w:i for i,w in enumerate(itos)})

In [ ]:
trn_ids = np.array([([stoi[w] for w in s]) for s in trn_tok])
val_ids = np.array([([stoi[w] for w in s]) for s in val_tok])
tst_ids = np.array([([stoi[w] for w in s]) for s in tst_tok])

## Testing WeightDropout

Create a bunch of parameters for deterministic tests.

In [ ]:
module = nn.LSTM(20, 20)
tst_input = torch.randn(2,5,20)
tst_output = torch.randint(0,20,(10,)).long()
save_params = {}
for n,p in module._parameters.items(): save_params[n] = p.clone()

### Old WeightDropout

In [ ]:
module = nn.LSTM(20, 20)
for n,p in save_params.items(): module._parameters[n] = nn.Parameter(p.clone())
dp_module = WeightDrop(module, 0.5)
opt = optim.SGD(dp_module.parameters(), 10)
dp_module.train()

In [ ]:
torch.manual_seed(7)

In [ ]:
x = tst_input.clone()
x.requires_grad_(requires_grad=True)
h = (torch.zeros(1,5,20), torch.zeros(1,5,20))
for _ in range(5): x,h = dp_module(x,h)

In [ ]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module.module,'weight_hh_l0_raw')

In [ ]:
target = tst_output.clone()
loss = F.nll_loss(x.view(-1,20), target)
loss.backward()
opt.step()

In [ ]:
w, w_raw = getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module.module,'weight_hh_l0_raw')
w.grad, w_raw.grad

In [ ]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module.module,'weight_hh_l0_raw')

### New WeightDropout

In [ ]:
class WeightDropout(nn.Module):
    "A module that warps another layer in which some weights will be replaced by 0 during training."
    
    def __init__(self, module, dropout, layer_names=['weight_hh_l0']):
        super().__init__()
        self.module,self.dropout,self.layer_names = module,dropout,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
    
    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.dropout, training=self.training)
            
    def forward(self, *args):
        self._setweights()
        return self.module.forward(*args)
    
    def reset(self):
        if hasattr(self.module, 'reset'): self.module.reset()

In [ ]:
module = nn.LSTM(20, 20)
for n,p in save_params.items(): module._parameters[n] = nn.Parameter(p.clone())
dp_module = WeightDropout(module, 0.5)
opt = optim.SGD(dp_module.parameters(), 10)
dp_module.train()

In [ ]:
torch.manual_seed(7)

In [ ]:
x = tst_input.clone()
x.requires_grad_(requires_grad=True)
h = (torch.zeros(1,5,20), torch.zeros(1,5,20))
for _ in range(5): x,h = dp_module(x,h)

In [ ]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')

In [ ]:
target = tst_output.clone()
loss = F.nll_loss(x.view(-1,20), target)
loss.backward()
opt.step()

In [ ]:
w, w_raw = getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')
w.grad, w_raw.grad

In [ ]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')

## Testing EmbeddingDropout

Create a bunch of parameters for deterministic tests.

In [ ]:
enc = nn.Embedding(100,20, padding_idx=0)
tst_input = torch.randint(0,100,(25,)).long()
save_params = enc.weight.clone()

### Old EmbeddingDropout

In [ ]:
enc = nn.Embedding(100,20, padding_idx=0)
enc.weight = nn.Parameter(save_params.clone())
enc_dp = EmbeddingDropout(enc)

In [ ]:
torch.manual_seed(7)

In [ ]:
x = tst_input.clone()
enc_dp(x, dropout=0.5)

### New EmbeddingDropout

In [ ]:
def dropout_mask(x, sz, p):
    "Returns a dropout mask of the same type as x, size sz, with probability p to cancel an element."
    return x.new(*sz).bernoulli_(1-p)/(1-p)

In [ ]:
class EmbeddingDropout1(nn.Module):

    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    def __init__(self, emb):
        super().__init__()
        self.emb = emb
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1

    def forward(self, words, dropout=0.1, scale=None):
        if dropout:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, dropout)
            masked_emb_weight = mask * self.emb.weight
        else: masked_emb_weight = self.emb.weight
        if scale: masked_emb_weight = scale * masked_emb_weight
        return F.embedding(words, masked_emb_weight, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [ ]:
enc = nn.Embedding(100,20, padding_idx=0)
enc.weight = nn.Parameter(save_params.clone())
enc_dp = EmbeddingDropout1(enc)

In [ ]:
torch.manual_seed(7)

In [ ]:
x = tst_input.clone()
enc_dp(x, dropout=0.5)

## Testing RNN model

Creating a bunch of parameters for deterministic testing.

In [ ]:
tst_model = get_language_model(500, 20, 100, 2, 0, bias=True)
save_parameters = {}
for n,p in tst_model.state_dict().items(): save_parameters[n] = p.clone()
tst_input = torch.randint(0, 500, (10,5)).long()
tst_output = torch.randint(0, 500, (50,)).long()

### Old RNN model

In [ ]:
tst_model = get_language_model(500, 20, 100, 2, 0, bias=True)
state_dict = OrderedDict()
for n,p in save_parameters.items(): state_dict[n] = p.clone()
tst_model.load_state_dict(state_dict)
opt = optim.SGD(tst_model.parameters(), lr=10)

In [ ]:
torch.manual_seed(7)

In [ ]:
x = tst_input.clone()
z = tst_model(x)
z

In [ ]:
y = tst_output.clone()
loss = F.nll_loss(z[0], y)
loss.backward()
opt.step()

In [ ]:
tst_model[0].rnns[0].module._parameters['weight_hh_l0_raw']

### New RNN model

In [ ]:
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p=p

    def forward(self, x):
        if not self.training or not self.p: return x
        m = dropout_mask(x.data, (1, x.size(1), x.size(2)), self.p)
        return m * x

In [ ]:
def repackage_var1(h):
    "Detaches h from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(repackage_var(v) for v in h)

In [ ]:
class RNNCore(nn.Module):
    "AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182"

    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token, bidir=False,
                 dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5, qrnn=False):
        
        super().__init__()
        self.bs,self.qrnn,self.ndir = 1, qrnn,(2 if bidir else 1)
        self.emb_sz,self.n_hid,self.n_layers,self.dropoute = emb_sz,n_hid,n_layers,dropoute
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.dp_encoder = EmbeddingDropout(self.encoder)
        if self.qrnn:
            #Using QRNN requires cupy: https://github.com/cupy/cupy
            from .torchqrnn.qrnn import QRNNLayer
            self.rnns = [QRNNLayer(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                save_prev_x=True, zoneout=0, window=2 if l == 0 else 1, output_gate=True) for l in range(n_layers)]
            if wdrop:
                for rnn in self.rnns:
                    rnn.linear = WeightDropout(rnn.linear, wdrop, layer_names=['weight'])
        else:
            self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                1, bidirectional=bidir) for l in range(n_layers)]
            if wdrop: self.rnns = [WeightDropout(rnn, wdrop) for rnn in self.rnns]
        self.rnns = torch.nn.ModuleList(self.rnns)
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.dropouti = RNNDropout(dropouti)
        self.dropouths = nn.ModuleList([RNNDropout(dropouth) for l in range(n_layers)])

    def forward(self, input):
        sl,bs = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        emb = self.dp_encoder(input, dropout=self.dropoute if self.training else 0)
        emb = self.dropouti(emb)
        raw_output = emb
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,drop) in enumerate(zip(self.rnns, self.dropouths)):
            #with warnings.catch_warnings():
            #    warnings.simplefilter("ignore")
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = drop(raw_output)
            outputs.append(raw_output)
        self.hidden = repackage_var1(new_hidden)
        return raw_outputs, outputs

    def one_hidden(self, l):
        nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz)//self.ndir
        return self.weights.new(self.ndir, self.bs, nh).zero_()

    def reset(self):
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.weights = next(self.parameters()).data
        if self.qrnn: self.hidden = [self.one_hidden(l) for l in range(self.n_layers)]
        else: self.hidden = [(self.one_hidden(l), self.one_hidden(l)) for l in range(self.n_layers)]

In [ ]:
class LinearDecoder1(nn.Module):
    "To go on top of a RNN_Core module"
    
    initrange=0.1
    
    def __init__(self, n_out, n_hid, dropout, tie_encoder=None, bias=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.dropout = RNNDropout(dropout)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.dropout(outputs[-1])
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [ ]:
class SequentialRNN1(nn.Sequential):
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [ ]:
def get_language_model1(vocab_sz, emb_sz, n_hid, n_layers, pad_token, tie_weights=True, qrnn=False, bias=True,
                 dropout=0.4, dropouth=0.3, dropouti=0.5, dropoute=0.1, wdrop=0.5):
    "To create a full AWD-LSTM"
    rnn_enc = RNNCore(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token, qrnn=qrnn,
                 dropouth=dropouth, dropouti=dropouti, dropoute=dropoute, wdrop=wdrop)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN1(rnn_enc, LinearDecoder1(vocab_sz, emb_sz, dropout, tie_encoder=enc, bias=bias))

The new model has weights that are organized a bit differently.

In [ ]:
save_parameters1 = {}
for n,p in save_parameters.items(): 
    if 'weight_hh_l0' not in n and n!='0.encoder_with_dropout.embed.weight':  save_parameters1[n] = p.clone()
    elif n=='0.encoder_with_dropout.embed.weight': save_parameters1['0.dp_encoder.embed.weight'] = p.clone()
    else: 
        save_parameters1[n[:-4]] = p.clone()
        splits = n.split('.')
        splits.remove(splits[-2])
        n1 = '.'.join(splits)
        save_parameters1[n1] = p.clone()

In [ ]:
tst_model = get_language_model1(500, 20, 100, 2, 0)
tst_model.load_state_dict(save_parameters1)
opt = optim.SGD(tst_model.parameters(), lr=10)

In [ ]:
torch.manual_seed(7)

In [ ]:
x = tst_input.clone()
z = tst_model(x)
z

In [ ]:
y = tst_output.clone()
loss = F.nll_loss(z[0], y)
loss.backward()
opt.step()

In [ ]:
tst_model[0].rnns[0]._parameters['weight_hh_l0_raw']

## Regularization

We'll keep the same param as before.

### Old reg

In [ ]:
tst_model = get_language_model(500, 20, 100, 2, 0, bias=True)
state_dict = OrderedDict()
for n,p in save_parameters.items(): state_dict[n] = p.clone()
tst_model.load_state_dict(state_dict)
opt = optim.SGD(tst_model.parameters(), lr=10, weight_decay=1)

In [ ]:
torch.manual_seed(7)

In [ ]:
x = tst_input.clone()
z = tst_model(x)
y = tst_output.clone()
loss = F.nll_loss(z[0], y)

In [ ]:
loss = seq2seq_reg(z[0], z[1:], loss, 2, 1)
loss.item()

In [ ]:
loss.backward()
nn.utils.clip_grad_norm_(tst_model.parameters(), 0.1)
opt.step()

In [ ]:
tst_model[0].rnns[0].module._parameters['weight_hh_l0_raw']

### New reg

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class RNNTrainer(Callback):
    model:nn.Module
    bptt:int
    clip:float=None
    alpha:float=0.
    beta:float=0.
    
    def on_loss_begin(self, last_output, **kwargs):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = last_output[1],last_output[2]
        return last_output[0]
    
    def on_backward_begin(self, last_loss, last_input, last_output, **kwargs):
        #Adjusts the lr to the bptt selected
        #self.learn.opt.lr *= last_input.size(0) / self.bptt
        #AR and TAR
        if self.alpha != 0.:  last_loss += (self.alpha * self.out[-1].pow(2).mean()).sum()
        if self.beta != 0.:
            h = self.raw_out[-1]
            if len(h)>1: last_loss += (self.beta * (h[1:] - h[:-1]).pow(2).mean()).sum()
        return last_loss
    
    def on_backward_end(self, **kwargs):
        if self.clip:  nn.utils.clip_grad_norm_(self.model.parameters(), self.clip)

In [ ]:
save_parameters1 = {}
for n,p in save_parameters.items(): 
    if 'weight_hh_l0' not in n and n!='0.encoder_with_dropout.embed.weight':  save_parameters1[n] = p.clone()
    elif n=='0.encoder_with_dropout.embed.weight': save_parameters1['0.dp_encoder.embed.weight'] = p.clone()
    else: 
        save_parameters1[n[:-4]] = p.clone()
        splits = n.split('.')
        splits.remove(splits[-2])
        n1 = '.'.join(splits)
        save_parameters1[n1] = p.clone()

In [ ]:
tst_model = get_language_model1(500, 20, 100, 2, 0)
tst_model.load_state_dict(save_parameters1)
opt = optim.SGD(tst_model.parameters(), lr=10, weight_decay=1)

In [ ]:
torch.manual_seed(7)

In [ ]:
cb = RNNTrainer(tst_model, 10, 0.1, 2, 1)

In [ ]:
x = tst_input.clone()
z = tst_model(x)
y = tst_output.clone()
z = cb.on_loss_begin(z)
loss = F.nll_loss(z, y)
loss = cb.on_backward_begin(loss, x, z)
loss.item()

In [ ]:
loss.backward()
cb.on_backward_end()
opt.step()

In [ ]:
tst_model[0].rnns[0]._parameters['weight_hh_l0_raw']